In [ ]:
import yaml
import os    
import scipy.io.wavfile
import numpy as np



# md := metadata
# ds := dataset
# op := output

dirroot_md = 'Metadata'
dirroot_ds = 'V2'
dirroot_op = 'Output'

##########################
# Create dirroot_op
##########################

if os.path.exists(dirroot_op) == False:
    os.mkdir(dirroot_op)



##########################
# iterate thru files in metadata dir
##########################

for filename_md in os.listdir(dirroot_md):
    
    # Open yaml metadata file
    filepath_md = os.path.join(dirroot_md, filename_md)
    stream = open(filepath_md)
    loaded = yaml.safe_load(stream)
    
    # Check if ds dir exists
    songname = filename_md.replace('_METADATA.yaml','')
    if os.path.exists(os.path.join(dirroot_ds, songname)) == False:
        continue;
    
    # Create op dir
    dirpath_op = os.path.join(dirroot_op, songname)
    if os.path.exists(dirpath_op) == False:
        os.mkdir(dirpath_op)    
        
    
    #############
    # Get mixture
    #############
    
    # get ds path
    mixname_ds = loaded.get('mix_filename')
    mixpath_ds = os.path.join(dirroot_ds, songname, mixname_ds)
    
    # read ds file
    sample_rate_mix, mixture = scipy.io.wavfile.read(mixpath_ds)
    
    # save op file
    scipy.io.wavfile.write(os.path.join(dirpath_op, 'mixture.wav'), sample_rate_mix, mixture)
    
    
    #############
    # Get vocals
    #############
    
    # get an empty vocals
    vocals = np.zeros(mixture.shape)
    
    # iterate through the stems
    stems = loaded.get('stems')
    stems_dir = loaded.get('stem_dir')
    num_vocal_stems = 0
    
    for key, value in stems.items():
        
        instrument = value.get('instrument')
        
        # found a vocals stem
        if 'vocalist' in instrument or 'singer' in instrument:
            
            num_vocal_stems += 1
            
            # get ds path
            volname_ds = value.get('filename')
            volpath_ds = os.path.join(dirroot_ds, songname, stems_dir, volname_ds)   
            
            # read ds file
            sample_rate_vol, data_vol = scipy.io.wavfile.read(volpath_ds)
            if (num_vocal_stems == 1):
                vocals = data_vol
            else:
                vocals = np.add(vocals, data_vol)

    
    # scale if needed
    vocals_max = np.amax(np.absolute(vocals))
    if vocals_max >= 32768:
        vocals *= 32768/vocals_max
    
    # save op file
    if (num_vocal_stems > 0):
        scipy.io.wavfile.write(os.path.join(dirpath_op, 'vocals.wav'), sample_rate_mix, vocals)